## Setting up the Dataset

We will only need pandas and numpy for this exercise

In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm

Read in the parquet file

In [2]:
df = pd.read_parquet('retail_flow_daily.parquet')

Preview of the data

In [3]:
print(df.head())

         date ticker  volume_buy  volume_sell  vwap_buy  vwap_sell
0  2016-01-04      A       28418        30233     40.60      40.54
1  2016-01-04    AAL      448009       437415     40.79      40.83
2  2016-01-04    AAP       45070        31598    152.29     152.02
3  2016-01-04   AAPL     3030463      2640489    103.91     103.84
4  2016-01-04   ABBV      307619       291025     57.25      57.28


Sort values by ticker, and then by date

In [4]:
df = df.sort_values(by=['ticker', 'date'])
df = df.reset_index(drop=True)

Create a column full of 1's. This will allow us to easily see how many days have passed (since we'll be looking at previous x trading days.

In [5]:
df['ones']= 1

Index by ticker and date, and then cumulative sum the data

In [6]:
df_grouped = df.groupby(['ticker', 'date']).sum().groupby(level=0).cumsum()

In [7]:
print(df_grouped)

                   volume_buy  volume_sell   vwap_buy  vwap_sell  ones
ticker date                                                           
A      2016-01-04       28418        30233      40.60      40.54     1
       2016-01-05       47714        74909      81.21      81.09     2
       2016-01-06       68063       103797     121.78     121.78     3
       2016-01-07      109157       142496     161.03     161.00     4
       2016-01-08      141344       168150     199.86     199.87     5
...                       ...          ...        ...        ...   ...
ZTS    2022-01-06    68433030     64925805  161743.90  161727.86  1515
       2022-01-07    68470994     64967654  161956.81  161940.81  1516
       2022-01-10    68516628     65009491  162167.53  162151.30  1517
       2022-01-11    68562959     65049639  162378.97  162362.67  1518
       2022-01-12    68603187     65085979  162591.86  162575.38  1519

[737378 rows x 5 columns]


In [8]:
df_grouped = df_grouped.reset_index(drop=True)

Factor 1: percent change over the last 20 trading days relative to prior 20 trading day period.

In [9]:
factor1 = []

In [10]:
for idx in df_grouped.index:
    if df_grouped['ones'][idx] >= 40:
        cur_buy_change = df_grouped['volume_buy'][idx] - df_grouped['volume_buy'][idx-19]
        prev_buy_change = df_grouped['volume_buy'][idx-20] - df_grouped['volume_buy'][idx-39]
        percent_change = (cur_buy_change - prev_buy_change) / prev_buy_change
        factor1.append(percent_change)
    else:
        factor1.append(None)

Factor 2: Total volume buys in the last 20 trading days

In [11]:
factor2 = []

In [12]:
for idx in df_grouped.index:
    if df_grouped['ones'][idx] >= 20:
        l20_volume = df_grouped['volume_buy'][idx] - df_grouped['volume_buy'][idx-19]
        factor2.append(l20_volume)
    else:
        factor2.append(None)

Add factor1, factor 2, and the number of days column to the original dataframe, since the third factor doesn't need cumulative sum data.

In [13]:
df['trading_days'] = df_grouped['ones']
df['factor_1'] = factor1
df['factor_2'] = factor2

In [14]:
print(df)

              date ticker  volume_buy  volume_sell  vwap_buy  vwap_sell  ones  \
0       2016-01-04      A       28418        30233     40.60      40.54     1   
1       2016-01-05      A       19296        44676     40.61      40.55     1   
2       2016-01-06      A       20349        28888     40.57      40.69     1   
3       2016-01-07      A       41094        38699     39.25      39.22     1   
4       2016-01-08      A       32187        25654     38.83      38.87     1   
...            ...    ...         ...          ...       ...        ...   ...   
737373  2022-01-06    ZTS       51517        38158    218.14     217.93     1   
737374  2022-01-07    ZTS       37964        41849    212.91     212.95     1   
737375  2022-01-10    ZTS       45634        41837    210.72     210.49     1   
737376  2022-01-11    ZTS       46331        40148    211.44     211.37     1   
737377  2022-01-12    ZTS       40228        36340    212.89     212.71     1   

        trading_days  facto

Factor 3: Cosine similarity of monthly volume buys

In [15]:
factor3 = []

In [16]:
for idx in df.index:
    if df['trading_days'][idx] >= 40:
        cur20 = df.loc[(idx-19):idx, ['volume_buy']].to_numpy().flatten()
        prev20 = df.loc[(idx-39):(idx-20), ['volume_buy']].to_numpy().flatten()
        cosine = np.dot(cur20,prev20)/(norm(cur20)*norm(prev20))
        
        dif = np.sum(cur20) - np.sum(prev20)
        
        if dif >= 0:
            direction = 1
        else:
            direction = -1
        
        factor3.append((cosine * direction))
    else:
        factor3.append(None)

In [17]:
df['factor_3'] = factor3

In [18]:
print(df)

              date ticker  volume_buy  volume_sell  vwap_buy  vwap_sell  ones  \
0       2016-01-04      A       28418        30233     40.60      40.54     1   
1       2016-01-05      A       19296        44676     40.61      40.55     1   
2       2016-01-06      A       20349        28888     40.57      40.69     1   
3       2016-01-07      A       41094        38699     39.25      39.22     1   
4       2016-01-08      A       32187        25654     38.83      38.87     1   
...            ...    ...         ...          ...       ...        ...   ...   
737373  2022-01-06    ZTS       51517        38158    218.14     217.93     1   
737374  2022-01-07    ZTS       37964        41849    212.91     212.95     1   
737375  2022-01-10    ZTS       45634        41837    210.72     210.49     1   
737376  2022-01-11    ZTS       46331        40148    211.44     211.37     1   
737377  2022-01-12    ZTS       40228        36340    212.89     212.71     1   

        trading_days  facto

Add in the pricing data to the dataset

In [19]:
px_data = pd.read_parquet('equity_prices.parquet')

In [20]:
px_data = px_data.sort_values(by=['ticker', 'date'])
px_data = px_data.reset_index(drop=True)
px_data = px_data.drop_duplicates(subset=['ticker', 'date'])

In [21]:
df['date'] =  pd.to_datetime(df['date'])
px_data['date'] =  pd.to_datetime(px_data['date'])
df = pd.merge(df, px_data, how='left', on=['ticker', 'date'])

In [22]:
print(df.head())

        date ticker  volume_buy  volume_sell  vwap_buy  vwap_sell  ones  \
0 2016-01-04      A       28418        30233     40.60      40.54     1   
1 2016-01-05      A       19296        44676     40.61      40.55     1   
2 2016-01-06      A       20349        28888     40.57      40.69     1   
3 2016-01-07      A       41094        38699     39.25      39.22     1   
4 2016-01-08      A       32187        25654     38.83      38.87     1   

   trading_days  factor_1  factor_2  factor_3   open    high     low  close  \
0             1       NaN       NaN       NaN  41.06  41.190  40.340  40.69   
1             2       NaN       NaN       NaN  40.73  40.950  40.340  40.55   
2             3       NaN       NaN       NaN  40.24  40.990  40.050  40.73   
3             4       NaN       NaN       NaN  40.14  40.150  38.810  39.00   
4             5       NaN       NaN       NaN  39.22  39.705  38.465  38.59   

      volume  closeadj  closeunadj lastupdated  
0  3287335.0    38.614   

Add in the ticker metadata

In [23]:
metadata = pd.read_csv('tickers_and_metadata.csv')
metadata = metadata[['ticker', 'sector']]

In [24]:
df = pd.merge(df, metadata, how='left', on=['ticker'])

In [25]:
print(df)

              date ticker  volume_buy  volume_sell  vwap_buy  vwap_sell  ones  \
0       2016-01-04      A       28418        30233     40.60      40.54     1   
1       2016-01-04      A       28418        30233     40.60      40.54     1   
2       2016-01-05      A       19296        44676     40.61      40.55     1   
3       2016-01-05      A       19296        44676     40.61      40.55     1   
4       2016-01-06      A       20349        28888     40.57      40.69     1   
...            ...    ...         ...          ...       ...        ...   ...   
1462600 2022-01-10    ZTS       45634        41837    210.72     210.49     1   
1462601 2022-01-11    ZTS       46331        40148    211.44     211.37     1   
1462602 2022-01-11    ZTS       46331        40148    211.44     211.37     1   
1462603 2022-01-12    ZTS       40228        36340    212.89     212.71     1   
1462604 2022-01-12    ZTS       40228        36340    212.89     212.71     1   

         trading_days  fact

In [26]:
df = df.drop_duplicates()

In [27]:
df = df[df['ticker'] != 'DISCA']
df = df[df['ticker'] != 'DOW']
df = df[df['ticker'] != 'FOX']
df = df[df['ticker'] != 'FOXA']
df = df[df['ticker'] != 'IR']
df = df[df['ticker'] != 'UA']

df = df.reset_index(drop=True)

## Portfolio Selection

Function for selecting the top X% of stocks based on a given factor. We select the top X% of stocks in each sector so that the resulting portfolio can be sector neutral. The resulting list will be the list of stocks to go long.

In [28]:
def portfolio_selector_longs(df, day, factor, percentage=.20):
    df_day = df[df['date'] == day]
    sectors = df_day['sector'].unique()
    
    array_list = []
    
    for sector in sectors:
        df_day_sector = df_day[df_day['sector'] == sector]
        
        n = round(df_day_sector['ticker'].nunique() * percentage)
        
        df_day_sector_largest = df_day_sector.nlargest(n=n, columns = [factor])
        
        tickers = df_day_sector_largest['ticker'].unique()
        
        array_list.append(tickers)
    
    portfolio = np.concatenate(array_list)
    return portfolio 

Function for selecting the bottom X% of stocks based on a given factor. We select the bottom X% of stocks in each sector so that the resulting portfolio can be sector neutral. The resulting list will be the list of stocks to short.

In [29]:
def portfolio_selector_shorts(df, day, factor, percentage=.20):
    df_day = df[df['date'] == day]
    sectors = df_day['sector'].unique()
    
    array_list = []
    
    for sector in sectors:
        df_day_sector = df_day[df_day['sector'] == sector]
        
        n = round(df_day_sector['ticker'].nunique() * percentage)
        
        df_day_sector_largest = df_day_sector.nsmallest(n=n, columns = [factor])
        
        tickers = df_day_sector_largest['ticker'].unique()
        
        array_list.append(tickers)
    
    portfolio = np.concatenate(array_list)
    return portfolio

Calculating the returns of a list of stocks to short. The idea behind a short is that you think the stock is overvalued and likely to drop in price. You borrow a stock, immediately sell it, and purchase it (hopefully) at a lower price to return back. Selling date is the start date, and ending date is the purchase date.

In [36]:
def calculate_returns_shorts(df, tickers, sell_date, purchase_date):
    df_sdate = df[df['date']==sell_date]
    df_pdate = df[df['date']==purchase_date]
    
    sell_prices = []
    purchase_prices = []
    for ticker in tickers:
        df_pdate_ticker = df_pdate[df_pdate['ticker']==ticker]
        df_pdate_ticker.reset_index(drop=True)
        
        df_sdate_ticker = df_sdate[df_sdate['ticker']==ticker]
        df_sdate_ticker.reset_index(drop=True)
        
        if df_pdate_ticker.empty or df_sdate_ticker.empty:
            sell_prices.append(float('nan'))
            purchase_prices.append(float('nan'))
        else:
            purchase_prices.append(df_pdate_ticker.iloc[0]['close'])
            sell_prices.append(df_sdate_ticker.iloc[0]['open'])
    
    stock_level_earnings = np.subtract(sell_prices, purchase_prices)
    stock_level_pct_earnings = np.divide(stock_level_earnings, sell_prices)
    
    return stock_level_pct_earnings

Calculating the returns of a list of stocks to go long. Purchase date is the start date, and the ending date is the selling date. We want the price at the selling date to be higher than the price at the purchase date.

In [37]:
def calculate_returns_longs(df, tickers, purchase_date, sell_date):
    df_pdate = df[df['date']==purchase_date]
    df_sdate = df[df['date']==sell_date]
    
    purchase_prices = []
    sell_prices = []
    for ticker in tickers:
        df_pdate_ticker = df_pdate[df_pdate['ticker']==ticker]
        df_pdate_ticker.reset_index(drop=True)
        
        df_sdate_ticker = df_sdate[df_sdate['ticker']==ticker]
        df_sdate_ticker.reset_index(drop=True)
        
        if df_pdate_ticker.empty or df_sdate_ticker.empty:
            purchase_prices.append(float('nan'))
            sell_prices.append(float('nan'))
        else:
            purchase_prices.append(df_pdate_ticker.iloc[0]['open'])
            sell_prices.append(df_sdate_ticker.iloc[0]['close'])
        
    stock_level_earnings = np.subtract(sell_prices, purchase_prices)
    stock_level_pct_earnings = np.divide(stock_level_earnings, purchase_prices)
    
    return stock_level_pct_earnings

Remove null values from the dataset for dates where a factor couldn't be produced since there weren't enough historical datapoints

In [54]:
df_filtered = df.dropna(subset=['factor_1', 'factor_2', 'factor_3'])

List of items to iterate through

In [66]:
dates = df_filtered['date'].unique()
holding_periods = [1, 2, 3, 4, 5, 10, 15, 20, 40, 60, 80]
percentages = [.05, .10, .15, .20]
factors = ['factor_1', 'factor_2', 'factor_3']

Create empty dataframe to append to

In [67]:
columns = ['factor', 'holding_period', 'cut_off', 'realized_return_date', 'side', 'return']
results_df = pd.DataFrame(columns=columns)

Iterate through each factor, holding period, and cut off, calculating each days return given the selected options for those variables.

In [68]:
for factor in factors:
    for holding_period in holding_periods:
        for cut_off in percentages:
            adj_holding_period = holding_period-1
            
            for idx in range(0, len(dates)-adj_holding_period-1):
                selection_date = dates[idx]
                start_date = dates[idx+1]
                end_date = dates[idx+adj_holding_period+1]
                
                longs = portfolio_selector_longs(df=df_filtered, day=selection_date,
                                                 factor=factor, percentage=cut_off)
                shorts = portfolio_selector_shorts(df=df_filtered, day=selection_date,
                                                   factor=factor, percentage=cut_off)
                
                longs_returns_pct = calculate_returns_longs(df=df_filtered, tickers=longs,
                                                            purchase_date = start_date, sell_date = end_date)
                shorts_returns_pct = calculate_returns_shorts(df=df_filtered, tickers=shorts,
                                                              sell_date = start_date, purchase_date = end_date)
                
                longs_return_total = np.mean(longs_returns_pct)
                shorts_return_total = np.mean(shorts_returns_pct)
                
                #add long dataframe
                factor_array = [factor]
                holding_period_array = [holding_period]
                cut_off_array = [cut_off]
                return_date_array = [end_date]
                side_array = ['long']
                
                dictionary_l = {'factor': factor_array, 'holding_period': holding_period_array,
                                'cut_off': cut_off_array, 'realized_return_date': return_date_array,
                                'side': side_array, 'return': longs_return_total}
                add_df = pd.DataFrame(data=dictionary_l)
                results_df = pd.concat([results_df, add_df], axis=0)
                
                #add short dataframe
                side_array = ['short']
                
                dictionary_s = {'factor': factor_array, 'holding_period': holding_period_array,
                                'cut_off': cut_off_array, 'realized_return_date': return_date_array,
                                'side': side_array, 'return': shorts_return_total}
                add_df = pd.DataFrame(data=dictionary_s)
                results_df = pd.concat([results_df, add_df], axis=0)

Printing results and resetting the index

In [69]:
print(results_df.reset_index(drop=True))

          factor holding_period  cut_off realized_return_date   side    return
0       factor_1              1     0.05           2016-03-02   long  0.012724
1       factor_1              1     0.05           2016-03-02  short -0.014056
2       factor_1              1     0.05           2016-03-03   long  0.006367
3       factor_1              1     0.05           2016-03-03  short -0.015051
4       factor_1              1     0.05           2016-03-04   long  0.004394
...          ...            ...      ...                  ...    ...       ...
384955  factor_3             80     0.20           2022-01-10  short -0.045816
384956  factor_3             80     0.20           2022-01-11   long  0.092658
384957  factor_3             80     0.20           2022-01-11  short -0.078080
384958  factor_3             80     0.20           2022-01-12   long  0.088462
384959  factor_3             80     0.20           2022-01-12  short -0.069448

[384960 rows x 6 columns]
